# Exploring the data
author: Nora Maleki (noora.maleki@gmail.com)
March 2022

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Load data

In [2]:
FILE_PATH = r'../../data/01_main'

In [3]:
dataset = pd.read_csv(f'{FILE_PATH}/01_coordinates_integrated.csv')
dataset.head()

,Unnamed: 0,SubjectID,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,Angle,BuildingPositionLocal_x,BuildingPositionLocal_y,BuildingPositionLocal_z,...,meaningful,meaningfulBuilding,Trials,TrialID,ImageName,TimeDelta,RT,TimeOut,TimeStampBegin,TimeStampEnd
0,0,365,-33.362625,-190.912018,-33.590775,-190.709625,87.005028,255.062286,-1.663044,-18.984482,...,True,Meaningful,0,29,01_CmA,0 days 00:00:07.635420416,7.635421,False,2021-11-18 10:26:39.660960000,2021-11-18 10:26:47.296380416
1,1,365,118.412003,-62.941864,118.647705,-63.044891,44.100197,255.062286,-1.663044,-18.984482,...,True,Meaningful,0,102,01_CmA,0 days 00:00:03.853917952,3.853918,False,2021-11-18 10:34:18.716167680,2021-11-18 10:34:22.570085632
2,2,365,257.929993,-33.097687,257.598694,-32.840240,47.522392,255.062286,-1.663044,-18.984482,...,True,Meaningful,0,155,01_CmA,0 days 00:00:03.418430208,3.418430,False,2021-11-18 10:40:28.512252160,2021-11-18 10:40:31.930682368
3,3,365,-242.933212,39.037964,-242.879013,38.953239,1.100918,255.062286,-1.663044,-18.984482,...,True,Meaningful,0,223,01_CmA,0 days 00:00:04.257166592,4.257167,False,2021-11-18 10:47:55.246826496,2021-11-18 10:47:59.503993088
4,4,365,-320.974487,77.197166,-320.853760,77.009575,29.176662,255.062286,-1.663044,-18.984482,...,True,Meaningful,0,303,01_CmA,0 days 00:00:02.474542336,2.474542,False,2021-11-18 11:08:19.997678080,2021-11-18 11:08:22.472220416


In [4]:
dataset.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3024 entries, 0 to 3023
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                3024 non-null   int64  
 1   SubjectID                 3024 non-null   int64  
 2   ParticipantPosition_x     3024 non-null   float64
 3   ParticipantPosition_z     3024 non-null   float64
 4   PointerPosition_x         3024 non-null   float64
 5   PointerPosition_z         3024 non-null   float64
 6   Angle                     3024 non-null   float64
 7   BuildingPositionLocal_x   3024 non-null   float64
 8   BuildingPositionLocal_y   3024 non-null   float64
 9   BuildingPositionLocal_z   3024 non-null   float64
 10  PointingTargetId          3024 non-null   int64  
 11  PointingTargetIndex       3024 non-null   int64  
 12  PointingTargetName        3024 non-null   object 
 13  PointingTargetBuildingId  3024 non-null   int64  
 14  Pointing

In [6]:
dataset.drop(columns=['Unnamed: 0', 'AvatarPresence'], inplace=True)

In [7]:
print(dataset.SubjectID.unique())

[ 365 3572 4176 5741 6642 7264 7842 8469 9472]


In [8]:
dataset

,SubjectID,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,Angle,BuildingPositionLocal_x,BuildingPositionLocal_y,BuildingPositionLocal_z,PointingTargetId,...,meaningful,meaningfulBuilding,Trials,TrialID,ImageName,TimeDelta,RT,TimeOut,TimeStampBegin,TimeStampEnd
0,365,-33.362625,-190.912018,-33.590775,-190.709625,87.005028,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,29,01_CmA,0 days 00:00:07.635420416,7.635421,False,2021-11-18 10:26:39.660960000,2021-11-18 10:26:47.296380416
1,365,118.412003,-62.941864,118.647705,-63.044891,44.100197,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,102,01_CmA,0 days 00:00:03.853917952,3.853918,False,2021-11-18 10:34:18.716167680,2021-11-18 10:34:22.570085632
2,365,257.929993,-33.097687,257.598694,-32.840240,47.522392,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,155,01_CmA,0 days 00:00:03.418430208,3.418430,False,2021-11-18 10:40:28.512252160,2021-11-18 10:40:31.930682368
3,365,-242.933212,39.037964,-242.879013,38.953239,1.100918,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,223,01_CmA,0 days 00:00:04.257166592,4.257167,False,2021-11-18 10:47:55.246826496,2021-11-18 10:47:59.503993088
4,365,-320.974487,77.197166,-320.853760,77.009575,29.176662,255.062286,-1.663044,-18.984482,1,...,True,Meaningful,0,303,01_CmA,0 days 00:00:02.474542336,2.474542,False,2021-11-18 11:08:19.997678080,2021-11-18 11:08:22.472220416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3019,7842,-242.936417,39.360191,-242.767288,39.562004,5.586337,-431.064392,12.052597,-241.219849,56,...,False,Not meaningful,110,308,56_Sa,0 days 00:00:05.227837952,5.227838,False,2021-12-10 11:26:34.675234816,2021-12-10 11:26:39.903072768
3020,7842,29.996002,177.688263,30.368515,178.135193,89.111778,-431.064392,12.052597,-241.219849,56,...,False,Not meaningful,110,325,56_Sa,0 days 00:00:17.154151168,17.154151,False,2021-12-10 11:27:39.271762432,2021-12-10 11:27:56.425913600
3021,8469,216.959259,133.368546,216.656937,133.552170,10.910424,-431.064392,12.052597,-241.219849,56,...,False,Not meaningful,110,146,56_Sa,0 days 00:00:12.459017472,12.459017,False,2021-11-12 14:55:31.406387712,2021-11-12 14:55:43.865405184
3022,8469,195.947968,-80.574738,195.926285,-80.244644,31.975525,-431.064392,12.052597,-241.219849,56,...,False,Not meaningful,110,249,56_Sa,0 days 00:00:08.456300032,8.456300,False,2021-11-12 15:17:15.624334848,2021-11-12 15:17:24.080634880


In [9]:
dataset.describe()

,SubjectID,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,Angle,BuildingPositionLocal_x,BuildingPositionLocal_y,BuildingPositionLocal_z,PointingTargetId,PointingTargetIndex,PointingTargetBuildingId,avatarID,Trials,TrialID,RT
count,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000,3024.000000
mean,5949.222222,-2.380455,14.179977,-2.105650,14.240982,49.312632,-0.507066,-0.480803,3.461617,28.521495,55.543651,29.076058,28.521495,55.543651,168.500000,7.694524
std,2679.017027,174.526182,137.780203,174.422929,137.616658,43.940893,201.542360,3.702814,128.275646,16.143405,32.306260,16.729943,16.143405,32.306260,97.010457,5.776413
min,365.000000,-326.886047,-191.062988,-327.157532,-191.119629,0.000000,-431.064392,-6.542114,-256.052216,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.955793
25%,4176.000000,-121.389214,-131.491875,-114.630356,-128.914780,13.787071,-153.392502,-2.357716,-91.287842,15.000000,28.000000,14.000000,15.000000,28.000000,84.750000,3.419546
50%,6642.000000,-15.471565,33.366211,1.953339,33.455299,36.979969,-3.306354,-1.083273,18.519926,29.000000,56.000000,29.000000,29.000000,56.000000,168.500000,5.872636
75%,7842.000000,123.511269,135.194962,137.915379,135.195560,72.340826,151.605972,0.727402,100.204834,43.000000,84.000000,43.000000,43.000000,84.000000,252.250000,9.962899
max,9472.000000,366.834778,235.882217,366.869934,235.829254,179.972031,401.828735,18.361389,293.247559,56.000000,111.000000,59.000000,56.000000,111.000000,336.000000,30.010961


In [29]:
all_buildings = pd.read_csv(f'{FILE_PATH}/02_all_buildings_list.csv')
all_buildings.head()

,Unnamed: 0,BuildingID,BuildingName,BuildingCenterWorld.x,BuildingCenterWorld.y,BuildingCenterWorld.z,BuildingPositionGlobal.x,BuildingPositionGlobal.y,BuildingPositionGlobal.z,BuildingPositionLocal.x,...,BuildingLocalScale.z,BuildingForwardDirection.x,BuildingForwardDirection.y,BuildingForwardDirection.z,BuildingUpDirection.x,BuildingUpDirection.y,BuildingUpDirection.z,BuildingRightDirection.x,BuildingRightDirection.y,BuildingRightDirection.z
0,0,16,Church-TaskBuilding_16,277.208374,19.870621,-91.080727,280.644775,-2.905334,-91.287842,280.644775,...,1.0,-3.576279e-07,-1.000000e+00,-2.980232e-07,-5.596234e-01,-5.215406e-08,8.287471e-01,0.828747,-4.172325e-07,0.559623
1,1,31,TaskBuilding_31,6.088593,5.046637,-0.542253,5.910362,-0.844291,-0.364021,5.910362,...,1.0,5.120487e-01,-3.606156e-07,-8.589562e-01,1.524359e-07,1.000000e+00,-3.289585e-07,-0.858956,-3.750693e-08,-0.512049
2,2,35,TaskBuilding_35,-72.578285,6.850483,53.288437,-72.798836,0.727402,53.811722,-72.798836,...,1.0,3.965120e-01,-4.116096e-07,-9.180295e-01,9.906053e-08,1.000000e+00,-4.055762e-07,-0.918030,-6.987533e-08,-0.396512
3,3,7,TaskBuilding_7,-192.461411,6.050417,-174.973267,-192.461411,-0.946640,-173.510040,-192.461411,...,1.0,4.611581e-01,1.051915e-07,8.873180e-01,1.513008e-08,1.000000e+00,-1.264134e-07,0.887318,-7.172174e-08,-0.461158
4,4,36,TaskBuilding_36,-51.411949,4.593466,-142.603455,-52.689445,-3.524445,-144.380814,-52.689445,...,1.0,-6.533048e-01,-2.793398e-07,-7.570951e-01,-1.038612e-07,1.000000e+00,-2.793398e-07,-0.757095,1.038612e-07,0.653305
